In [0]:
l = [(1,'Gaga','India',"2022-01-11"),(1,'Katy','UK', "2022-01-11"),(1,'Bey','Europe', "2022-01-11"),(2,'Gaga',None, "2022-10-11"),(2,'Katy','India', "2022-10-11"),(2,'Bey','US',"2022-02-15"),(3,'Gaga','Europe', "2022-10-11"),
(3,'Katy','US',"2022-10-11"),(3,'Bey',None,"2022-02-15"), (1, 'Gaga','US',"2022-01-11"),(3, 'Katy', 'Switz',"2022-02-15") ]
df = spark.createDataFrame(l,['ID','NAME','COUNTRY', 'Date_part'])
display(df)

ID,NAME,COUNTRY,Date_part
1,Gaga,India,2022-01-11
1,Katy,UK,2022-01-11
1,Bey,Europe,2022-01-11
2,Gaga,null,2022-10-11
2,Katy,India,2022-10-11
2,Bey,US,2022-02-15
3,Gaga,Europe,2022-10-11
3,Katy,US,2022-10-11
3,Bey,null,2022-02-15
1,Gaga,US,2022-01-11


In [0]:
from pyspark.sql.functions import *

In [0]:

df_pivot = df.groupBy(col('ID'), col('Date_part')).pivot('NAME').agg(collect_list(col('COUNTRY')))
display(df_pivot)

ID,Date_part,Bey,Gaga,Katy
1,2022-01-11,List(Europe),"List(India, US)",List(UK)
2,2022-10-11,List(),List(),List(India)
2,2022-02-15,List(US),List(),List()
3,2022-10-11,List(),List(Europe),List(US)
3,2022-02-15,List(),List(),List(Switz)


In [0]:
df_array=df_pivot.select('ID','Date_part',arrays_zip('Bey','Gaga','Katy').alias('new'))
display(df_array)

ID,Date_part,new
1,2022-01-11,"List(List(Europe, India, UK), List(null, US, null))"
2,2022-10-11,"List(List(null, null, India))"
2,2022-02-15,"List(List(US, null, null))"
3,2022-10-11,"List(List(null, Europe, US))"
3,2022-02-15,"List(List(null, null, Switz))"


In [0]:
df3=df_array.select('ID','Date_part',explode('new').alias('column'))
display(df3)

ID,Date_part,column
1,2022-01-11,"List(Europe, India, UK)"
1,2022-01-11,"List(null, US, null)"
2,2022-10-11,"List(null, null, India)"
2,2022-02-15,"List(US, null, null)"
3,2022-10-11,"List(null, Europe, US)"
3,2022-02-15,"List(null, null, Switz)"


In [0]:
df4=df3.select('column').rdd.map(lambda x: x[0]).collect()
display(df4)

Bey,Gaga,Katy
Europe,India,UK
null,US,null
null,null,India
US,null,null
null,Europe,US
null,null,Switz


In [0]:
df3.columns()

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-2940832757814414>:1
----> 1 df3.columns()

TypeError: 'list' object is not callable